## Notebook to calculate T1 anisotropy 
based on CSA   
using diffusion on a cone model   

In [1]:
# CSA parameters
iso_val = 170
delta_xx_val = 95
delta_yy_val = 190
delta_zz_val = 3 * iso_val - delta_xx_val - delta_yy_val  # 510 - 285 = 225
delta_val = iso_val -delta_zz_val  # 170 - 225 = -55

import numpy as np

# simulate a diffusion on a cone model 
def simulate_vector_on_cone(S2=0.85, tau_c=0.01, dt=1e-4, num_steps=10000, axis=np.array([0, 0, 1])):
    """
    Simulate a unit vector hopping on a cone surface with fixed S2 and correlation time tau_c.
    
    Returns:
        vectors: (num_steps, 3) array of unit vectors
    """
    # Cone angle from S²
    cos_theta = np.sqrt((2 * S2 + 1) / 3)
    theta = np.arccos(cos_theta)

    # Ornstein-Uhlenbeck parameters for azimuthal diffusion
    gamma = 1 / tau_c
    sigma = np.sqrt(2 * gamma)  # Unit noise strength
    phi = 0.0
    axis = axis / np.linalg.norm(axis)

    # Rotation matrix to align cone with axis
    R_align = rotation_matrix_from_vectors(np.array([0, 0, 1]), axis)

    vectors = np.zeros((num_steps, 3))

    for i in range(num_steps):
        # Update azimuthal angle using Ornstein-Uhlenbeck process
        dphi = -gamma * phi * dt + sigma * np.sqrt(dt) * np.random.randn()
        phi += dphi

        # Point on cone with fixed θ and current φ
        x = np.sin(theta) * np.cos(phi)
        y = np.sin(theta) * np.sin(phi)
        z = np.cos(theta)
        vec_local = np.array([x, y, z])

        # Rotate to align cone with specified axis
        vec_global = R_align @ vec_local
        vectors[i] = vec_global

    return vectors

def rotation_matrix_from_vectors(a, b):
    """Find the rotation matrix that aligns vector a to vector b"""
    a = a / np.linalg.norm(a)
    b = b / np.linalg.norm(b)
    v = np.cross(a, b)
    c = np.dot(a, b)
    if c == 1:
        return np.eye(3)
    if c == -1:
        # 180° rotation around arbitrary perpendicular axis
        perp = np.array([1, 0, 0]) if not np.allclose(a, [1, 0, 0]) else np.array([0, 1, 0])
        return rotation_matrix_from_vectors(a, np.cross(a, perp))
    s = np.linalg.norm(v)
    vx = np.array([[0, -v[2], v[1]],
                   [v[2], 0, -v[0]],
                   [-v[1], v[0], 0]])
    return np.eye(3) + vx + vx @ vx * ((1 - c) / (s**2))




In [2]:
# to make the trajactory T1 sensitive, simulate with 
vecs = simulate_vector_on_cone(S2=0.85, tau_c=1e-9, dt=5e-11, num_steps=2000)


In [3]:
# calculate autocorrelation function for spherical harmonics for all m values
from scipy.special import sph_harm

def compute_sph_harm_autocorrelation(vectors, l=2, m=0):
    """
    Compute complex autocorrelation of Y_l^m spherical harmonics from vector trajectory.
    
    Parameters:
        vectors: (num_steps, 3) array of unit vectors
        l: degree of spherical harmonics
        m: order of spherical harmonics

    Returns:
        corr: autocorrelation values
    """
    N = len(vectors)
    theta = np.arccos(vectors[:, 2])  # z = cos(theta)
    phi = np.arctan2(vectors[:, 1], vectors[:, 0])  # arctangent of y/x

    Y_vals = sph_harm(m, l, phi, theta)

    corr = []
    for tau in range(N // 10):
        if tau == 0:
            c = np.abs(np.mean(Y_vals * np.conj(Y_vals)))  # Normalization
        else:
            c = np.mean(Y_vals[:-tau] * np.conj(Y_vals[tau:]))
        corr.append(np.real(c))  # or keep complex values
    return np.array(corr)

def compute_correlation_matrix(Y_series, max_lag=1000):
    l = max(Y_series.keys())
    corr_matrix = {}
    for m1 in range(-l, l + 1):
        for m2 in range(-l, l + 1):
            corr = []
            y1 = Y_series[m1]
            y2 = Y_series[m2]
            for tau in range(max_lag//10):
                val = np.mean(y1[:-tau or None] * np.conj(y2[tau:])) if tau > 0 else np.mean(y1 * np.conj(y2))
                corr.append(val)
            corr_matrix[(m1, m2)] = np.array(corr)
    return corr_matrix

In [4]:
Y_series = {}
for m in range(-2, 3):
    Y_series[m] = compute_sph_harm_autocorrelation(vecs, l=2, m=m)
corr_matrix = compute_correlation_matrix(Y_series, max_lag=2000)


In [ ]:
# Number of random rotations
N = 100

# Sample random rotations from SO(3)
rot = R.random(num=N)

# Extract Euler angles (zyz convention)
alpha_vals, beta_vals, gamma_vals = rot.as_euler('zyz', degrees=False).T

# generate autocorrelation functions for m=-1 for each orientation

{(-2, -2): array([ 4.32493684e-08,  3.72943225e-08,  3.24248499e-08,  2.84324951e-08,
        2.51743613e-08,  2.25016799e-08,  2.04354554e-08,  1.88223897e-08,
        1.76618626e-08,  1.67200384e-08,  1.59359327e-08,  1.52944937e-08,
        1.47549318e-08,  1.43671298e-08,  1.39945772e-08,  1.36130198e-08,
        1.32174323e-08,  1.28392822e-08,  1.24372366e-08,  1.20307719e-08,
        1.16522747e-08,  1.13400525e-08,  1.10839743e-08,  1.07844117e-08,
        1.04448347e-08,  1.00875194e-08,  9.71656697e-09,  9.40247245e-09,
        9.18525414e-09,  9.02154534e-09,  8.88289748e-09,  8.67875748e-09,
        8.37372287e-09,  7.98101943e-09,  7.56869695e-09,  7.21016743e-09,
        6.97428169e-09,  6.79212730e-09,  6.63703410e-09,  6.56637649e-09,
        6.62520404e-09,  6.76811553e-09,  7.05467481e-09,  7.37638306e-09,
        7.73919006e-09,  8.19663229e-09,  8.67872421e-09,  9.15771822e-09,
        9.70513504e-09,  1.03399098e-08,  1.08513088e-08,  1.12653668e-08,
        1.1607

In [ ]:
# generate random orientations on a SO3 surface
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

# Number of random rotations
N = 100

# Sample random rotations from SO(3)
rot = R.random(num=N)

# Extract Euler angles (zyz convention)
alpha_vals, beta_vals, gamma_vals = rot.as_euler('zyz', degrees=False).T

# compute autocorrelation function for spherical harmonics m=-1 for each orientation using wigner D-matrix
# further calculate T1 value for that orientation


# Another way to compute autocorrelation using spherical harmonics
from scipy.special import sph_harm




# calculate chemical shift value for each orientation


# plot chemical shift vs T1 values